In [2]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import plotly.graph_objects as go
from scipy.stats import norm

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import get_mlp_activations
from hook_utils import get_ablate_neuron_hook
import haystack_utils
from pythia_160m_utils import get_neuron_accuracy, ablation_effect
from plotting_utils import plot_neuron_acts, color_binned_histogram


%reload_ext autoreload
%autoreload 2

In [3]:
model = HookedTransformer.from_pretrained("tiny-stories-33M",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device="cuda")

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-33M into HookedTransformer


In [4]:
dataset = load_dataset('roneneldan/TinyStories', split='train')
print(dataset[0])

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}


In [19]:
print(model.generate("John is high and Sarah is low. John is in a tree and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is in a treehouse and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on top of a pole and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on top of a house and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on the roof and Sarah", 100))


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is in a tree and Sarah is under his tree. He is stuck. John is scared. He says: "Help! Help!" Sarah hears him and moves off to another tree. John says: "Thank you, Sarah!" Sarah is happy to help.

But then Sarah hears a worried noise. She looks up and sees a bird cage. The cage is very high and Sarah's heart is beating very fast. She wants to help the bird. She quickly takes off the cushion and the cushage. She puts the


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is in a treehouse and Sarah is on the swing.

One day, they hear an alarm. John is scared and Sarah is curious. She asks John to do a test. John asked questions like, 'What is powerful?' and 'What should the alarm do?' Sarah thought for a minute and then she said 'I think it is because of the alarm'.

John felt much better and said to Sarah, “I think I understand now! Let’s find out who is causing the alarm and make


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on top of a pole and Sarah is on the ground. They both frowning and Sarah's frownss.

John was angry. He wanted to get down and shout. He thought of another way. He didn't want to stay up there with Sarah alone.

John had an idea. He took his scooter and pushed Sarah off the floor. Sarah fell on the floor with a loud clatter. She was mad.

John and Sarah started to quarrel. They yelled at each other and Sarah had to


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on top of a house and Sarah is close behind. Sarah is backing around and making noises.

"I want to have fun now," she says to John.

John is not happy. He said: "No, it's dangerous to go underneath the tree."
Sarah starts to cry.

But John mustered up the courage and said: "Come on, Sarah. Let's go under the tree."

So they both crawled under the tree and found a beautiful meadow full of flowers. They


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on the roof and Sarah is on the grass. John smiles and thinks this is a good idea. He makes a hole in the balloon and then he takes out a plastic stick. He cuts the string to make it. Sarah watches and asks why John does this when he stops climbing.

John looks back onto the ground. Sarah says, "Next time, I will help you climb the pile of leaves". John smiles. He likes to help Sarah. They work together and it is fun.
<|endoftext|>


In [21]:
print(model.generate("John is low and Sarah is high. John is on the ground and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is in a hole and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is underneath a pile of leave and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is under the house and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is under the horse and Sarah", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is on the ground and Sarah is on the ground. They are very sad. 

They see a big chain in the park. They both like chains. They want to pick it up and play with it.

They both pull on it and it breaks. They are very sad and the chain is gone. 

John and Sarah take the chain and go home. They hope to find the chain again someday. That was very special.
<|endoftext|>


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is in a hole and Sarah is not there.

"Where is Sarah?" John asked.

"I don't know," Sarah replied.

John kept searching and searching. He looked all around the hole, but he could not find Sarah.

"I am not in the hole!" he yelled out.

Suddenly a big purple spider appeared. It began to chase John. He ran and ran until he reached home.

The spider caught John and he was taken to the hospital. Sarah


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is underneath a pile of leave and Sarah is playing with her dolls. She sees John and smiles. She wants to be friends with John.

John's mom hears Sarah's voice and calls out "Emily, where are you?" She looks under the pile but she can not see Sarah.

Jane was happy to see Tim but she is still scared of his toes. Then she saw something on Sarah's toes. It was a bug! She stared at it. But then it scurries away.

Sarah said,


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is under the house and Sarah is still under it. Sarah cries and blinks her eyes. She is sad and misses John. She hopes that her pain will go away.

John hears Sarah crying. He feels bad for her and goes to her. He is slow and careful. He makes room for her out of the under. He takes her deeper from the hole and eats her candy. He makes her smile. He hopes that she will feel better soon.

Sarah's arm gets better after a few minutes.


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is under the horse and Sarah is under the horse. John is thirsty and the horse drinks from Sarah. Sarah is sad and she starts to cry.

Then suddenly, the horse stops eating. John is shocked and he wants to get Sarah out. But he does not know how to do it.

John is upset and he cries. Sarah feels bad about being left out. She is sorry that she should have been alone. Sarah needs to go back to the stable and John has to chase her.

Sarah


In [24]:
print(model.generate("John is low and Sarah is high. Sarah is up a tree and John is", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. Sarah is up a tree and John is at the bottom. He could see many things from up there. Sarah was on the swings and John could see them from up high. He was complaining because he was too small to reach them.

John kept complaining and Sarah looked away. Then she saw a man. He was carrying a big bag and looked easyening his way into the tree. When he got there he talked to mommy. He said he could get her over there because Sarah was too small.

Mommy thanked


In [29]:
print(model.generate("", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack and Sarah are in the meadow. One of them is in a tree. The other is a girl named Sophie.

John looks up and sees Sophie. He points at her and says, "She is teasing me!" Sarah ran to John and says, "Sorry, John. That's not nice at all."

John and Sarah climb the tree and sit on a branch. They look at the sky and the fluffy clouds that moved across the sky. Sarah pointed at the clouds and said, "Look John, I see a cloud that looks like a dog!"

John


In [56]:
# Selecting multi-use objects in the environment
print(model.generate("Jack was thirsty. Jack saw a pond, so he", 100, temperature=0))
print(model.generate("Jack was bored. Jack saw a pond, so he", 100, temperature=0))

print(model.generate("Jack was sleepy. Jack saw a mat, so he", 100, temperature=0))
print(model.generate("Jack was excited. Jack saw a mat, so he", 100, temperature=0))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack was thirsty. Jack saw a pond, so he went to take a drink. He saw a big frog and wanted to catch it. He tried to catch the frog, but it jumped away. Jack was sad, but he kept walking.

Suddenly, Jack saw a big dog. The dog barked at Jack and he got scared. He tried to run away, but the dog chased him. Jack ran and ran, but the dog was too fast. The dog caught Jack and bit him. Jack cried and cried.

The end


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was bored. Jack saw a pond, so he decided to go for a swim. He jumped in and started to splash around. He was having so much fun!

Suddenly, Jack heard a noise. He looked around and saw a big frog. The frog was jumping around and making a lot of noise. Jack was scared and he wanted to get out of the water.

Jack started to swim towards the frog. The frog saw Jack and started to swim away. Jack was sad and he started to cry.

Jack's mom


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was sleepy. Jack saw a mat, so he lay down on it. He closed his eyes and started to sleep.

Suddenly, Jack heard a noise. He opened his eyes and saw a big, scary monster! Jack was so scared he couldn't move.

The monster said, "I'm going to eat you!"

Jack was so scared he couldn't move. He just lay there, too scared to move.

The monster grabbed Jack and ate him up. Jack was never seen again.
<|endoftext|>


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was excited. Jack saw a mat, so he ran over to it. He picked it up and started to play. He rolled it around and laughed. He was having so much fun.

Suddenly, Jack heard a noise. He looked around and saw a big dog. The dog was barking and running towards him. Jack was scared. He tried to run away, but the dog was too fast.

The dog bit Jack's leg. Jack screamed and cried. He was hurt. The dog ran away.

Jack was sad


In [63]:
# multi-use environment elements
# print(model.generate("The ball rolled off the bridge", 100, temperature=0))
# print(model.generate("Put the ball onto the hill. The ball", 100, temperature=0))
# print(model.generate("Sarah put the box onto the hill. The box", 100, temperature=0))
# print(model.generate("The box was too big and heavy. Sarah pushed the box ", 100, temperature=0))




# print(model.generate("Jack was tired. Jack saw a slide, so he", 100, temperature=0))
# print(model.generate("Jack was thinking. Jack saw a slide, so he", 100, temperature=0))
# print(model.generate("Jack was near the pond, but he was thirsty so he ", 100, temperature=0))
# print(model.generate("Jack was near the pond, but he wasn't thirsty so he ", 100, temperature=0))
# print(model.generate("Jack was far away so he ", 100, temperature=0))
# print(model.generate("Jack was close by so he ", 100, temperature=0))
# print(model.generate("Jack was above Sarah so he ", 100, temperature=0))
# print(model.generate("Jack was a long way from home and he felt thirsty so he", 100, temperature=0))



  0%|          | 0/100 [00:00<?, ?it/s]

The box was too big and heavy. Sarah pushed the box and it started to roll. She was so happy! She ran after the box and it rolled down the street.

Sarah followed the box until it stopped at a park. She saw a big tree and she wanted to climb it. She put her foot on the trunk and started to climb. She was so excited!

But then she heard a loud noise. It was the park keeper. He was angry and he shouted at Sarah. He said she was not allowed to climb the tree


In [4]:
print(model.generate("Jack wanted to draw on the ground. Jack saw a stick. He ", 100, temperature=0))

print(model.generate("Jack wanted to draw on the ground. Jack saw a stick. He use ", 100, temperature=0))


  0%|          | 0/100 [00:00<?, ?it/s]

Jack wanted to draw on the ground. Jack saw a stick. He was very excited. He picked it up and started to draw on the ground. He drew a big circle and then a small square. He was very happy.

Jack's mom saw him and said, "Jack, what are you drawing?"

Jack said, "I'm drawing a circle. Look, it's so big and round!"

Jack's mom smiled and said, "That's great! You're so creative!"

Jack smiled and kept drawing. He was


In [60]:
print(model.generate("Once upon a time there lived a girl named Lily. She loved to look in her attic. One day, she found a pond in her attic. She ", 100, temperature=0))


  0%|          | 0/100 [00:00<?, ?it/s]

Once upon a time there lived a girl named Lily. She loved to look in her attic. One day, she found a pond in her attic. She was so excited to see it! She ran to the pond and saw that it was full of water. She wanted to play in it, but her mom said no.

Lily was sad, but then she had an idea. She asked her mom if she could soak her feet in the pond. Her mom said yes, so Lily put her feet in the water and it felt so nice. She splashed around and had so much fun.

After a while, Lily's mom


In [73]:
print(model.generate("Jack was playing in the park. But", 20, temperature=0))
print(model.generate("Jack was in the pool. But", 20, temperature=0))
print(model.generate("Jack was going to school. But", 20, temperature=0))
print(model.generate("Jack was sad. But", 20, temperature=0))
print(model.generate("Jack was sleeping in bed. But", 20, temperature=0))
print(model.generate("Jack was lying in bed. But", 20, temperature=0))

  0%|          | 0/20 [00:00<?, ?it/s]

Jack was playing in the park. But then he saw something strange. He saw a big, scary dog. It was barking and growling


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was in the pool. But he was not happy. He was very angry. He wanted to play with his friends, but they


  0%|          | 0/20 [00:00<?, ?it/s]

...

Jack was sad. But then he saw a big, shiny motorcycle. It was so cool! He wanted to ride it.


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was sleeping in bed. But he was not sleeping. He was dreaming about a big, scary monster. The monster was coming to


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was lying in bed. But he was not feeling very good. He was feeling very tired. He had been playing all day and


In [79]:
print(model.generate("Jack was a boy who loved to draw. He drew red drawings with his crayon and blue drawings with his pencil. He drew a blue circle with his", 20, temperature=0))
print(model.generate("Jack was a boy who loved to draw. He drew blue drawings with his crayon and red drawings with his pencil. He drew a blue circle with his", 20, temperature=0))


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was a boy who loved to draw. He drew red drawings with his crayon and blue drawings with his pencil. He drew a blue circle with his pencil and then drew a green triangle with his red pencil. He was very proud of his drawing.


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was a boy who loved to draw. He drew blue drawings with his crayon and red drawings with his pencil. He drew a blue circle with his pencil and then drew a green triangle with his blue pencil. He was very happy with his drawing.


In [22]:
print(model.generate(" The park was closed. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" The park was large. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" The park was very good. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a cat.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a dog.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a rock.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold an ant.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold an elephant.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a star.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a rocket.", 20, temperature=0))


# Basic properties of objects

  0%|          | 0/20 [00:00<?, ?it/s]

 The park was closed. Jack asked his mother whether he could go to the park. His mother said, "No, it's too late. You need to stay home." Jack was


  0%|          | 0/20 [00:00<?, ?it/s]

 The park was large. Jack asked his mother whether he could go to the park. His mother said, "Yes, but you must stay close to me."

Jack was so


  0%|          | 0/20 [00:00<?, ?it/s]

...

 Jack asked his mother whether he could hold an elephant. His mother said, "No, Jack. Elephants are too big and need to be in


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack asked his mother whether he could hold a star. His mother said, "No, Jack. Stars are too far away."

Jack was sad


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack asked his mother whether he could hold a rocket. His mother said, "No, Jack. Rockets are dangerous." Jack was sad, but he listened


In [64]:
# print(model.generate(" Jack had three apples. Jack ate one of his apples. Now Jack had", 20, temperature=0))

# print(model.generate(" Sarah had three pears. Sarah ate two pears. Then she had", 20, temperature=0))

# print(model.generate(" Jack picked up the apple but not left the star on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the rocket but he left the orange on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the orange but he left the rocket on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the star but he left the orange on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the orange but he left the apple on the ground. Jack held the", 20, temperature=0))

# print(model.generate(" Jack held the apple but not the star. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the rocket but not the orange. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the orange but not the rocket. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the star but not the orange. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the orange but not the apple. Jack held the", 20, temperature=0))


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the apple but not the star. Jack held the apple tight and said, "I don't want to share this apple with you."

His


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the rocket but not the orange. Jack held the rocket and it felt so light. He was so happy. He wanted to show his friends.



  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the orange but not the rocket. Jack held the rocket and it felt so light. He was so happy. He wanted to show his friends.



  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the star but not the orange. Jack held the star and looked at it. He was so happy. He wanted to show it to his mom.


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the orange but not the apple. Jack held the apple and looked at it. He wanted to eat it, but he knew he had to be careful


In [5]:
print(model.generate("Jack asked his mother whether he could have an icecream. His mother said", 100, temperature=0))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack asked his mother whether he could have an icecream. His mother said, "No, Jack. You need to eat something healthy first."

Jack was sad and he started to cry. His mother said, "Don't worry, Jack. I will make you an ice cream. It will be yummy and healthy."

Jack was happy and he stopped crying. He said, "Yes, please!"

His mother made him an ice cream and Jack ate it. He was so happy and he said, "Thank you, Mommy!"



In [44]:
# prompt = "Jack said, \"Can I have a frog?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack was going away and, \"Can I have a frog?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack am, \"Am I alive?.\""
# print(model.generate(prompt, 100, temperature=0))


# prompt = "Jack said, \"Why is red?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack said, \"Why is we alive?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack said, \"Why is the cow fast?.\""
# print(model.generate(prompt, 100, temperature=0))

prompt = "Jack said, \"why tree blue?.\""
print(model.generate(prompt, 100, temperature=0))




  0%|          | 0/100 [00:00<?, ?it/s]

Jack said, "why tree blue?."

His mom said, "The tree is green. It is a special tree."

Jack said, "I want to climb it!"

His mom said, "No, it is too high. You can't climb it."

Jack said, "But I want to climb it!"

His mom said, "No, it is too dangerous. You can't climb it."

Jack said, "But I want to climb it!"

His mom


In [65]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his mother Sally.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his mother Sally. He was so excited and he wanted to get it.

Jack asked his mother if he could have some ice cream. His mother said yes and she gave him a big bowl of ice cream. Jack was so happy and he ate it all up.

But then Jack started to feel sick. He had eaten too much ice cream and his tummy hurt. He started to cry and his mother was very worried. She took him to the hospital and the doctor said he had to stay in the hospital for a few days.

Jack was very sad and he was very sorry for not listening to his mother. He learned that it is important to listen to your parents and not eat too much ice cream.
<|endoftext|>


In [66]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Jack was so excited and he wanted to get the ice cream.

Jack asked Sarah if he could have some ice cream. Sarah said no and told Jack to go away. Jack was very sad and he started to cry.

Suddenly, a big man came and he was very angry. He said to Jack, "You must leave the park now!" Jack was scared and he ran away.

Jack was so sad that he didn't get the ice cream. He was very sad and he never got to eat the ice cream.
<|endoftext|>


In [69]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sally, but he had to go home.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sally, but he had to go home.

Jack was sad because he wanted to eat the ice cream. He asked Sally if he could have some, but she said no. Jack was very sad and he started to cry.

Suddenly, a big, scary man came and he was very angry. He said that Jack was not allowed to have any ice cream. He said that he would punish Jack if he didn't leave the park.

Jack was scared and he ran away. He was so sad that he didn't get any ice cream. He was very sad and he never went to the park again.
<|endoftext|>


In [67]:
print(model.generate(" Once upon a time there was a girl named Sarah and a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a girl named Sarah and a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. He asked Sarah if he could have some. Sarah said yes and gave Jack a big scoop of ice cream.

Jack was so happy and he wanted to show Sarah how much he liked the ice cream. He asked Sarah if he could have some more. Sarah said yes and gave Jack a big scoop of ice cream. Jack was so happy and he thanked Sarah for the ice cream.

Jack and Sarah had a lot of fun at the park. They played on the swings and the slide. They even had a picnic with some of the ice cream. Jack was so happy and he thanked Sarah for the ice cream.

Jack and Sarah had a great time at the park. They both agreed that it was the best day ever!
<|endoftext|>


In [68]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Sarah gave him some icecream. Jack was very happy. ", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Sarah gave him some icecream. Jack was very happy. 

Jack and Sarah went to the park and Jack saw a big tree. He wanted to climb it. He started to climb the tree but it was too hard. He was too scared. 

Then Sarah said, "Let's go to the ice cream shop. I will give you some ice cream." Jack was so happy. He said, "Thank you Sarah!" 

They went to the ice cream shop and Jack got his ice cream. He was so happy. He ate it all up. 

Jack and Sarah had a great day at the park. They were both very happy.
<|endoftext|>


In [72]:
print(model.generate(" Once upon a time, Jack and Sarah lived by a chair. One day, they both wanted to sit in the chair at the same time. They didn't want to sit together. Sarah sat in the chair. Jack ", 200, temperature=0))


  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time, Jack and Sarah lived by a chair. One day, they both wanted to sit in the chair at the same time. They didn't want to sit together. Sarah sat in the chair. Jack was getting angry. He said, "I want to sit in the chair!" Sarah said, "No, I want to sit in the chair!" They started to argue and it made them both very sad.

Suddenly, Jack had an idea. He said, "Let's both sit in the chair together!" Sarah thought that was a great idea. They both sat in the chair together and they were happy again. They realized that sharing was better than fighting. From that day on, they always shared the chair when they were together.
<|endoftext|>


In [87]:
continuous_prompts = [
    'Sarah was painting and',
    'Sarah was sleeping and dream',
    'Sarah was paint',
    'Sarah was painting and coloring over the white parts of her bedroom wall', 
    'Jack was swimming in the pool. He was going to drink the water but his mother was looking at him',
    'Jack and Sarah were playing in the yard',
    'Jack was eating an icecream when the teacher walked in',
    'They were watching a movie when the cat arrived',
    'Sarah was sleeping and dreaming of animals',
    'Sarah was playing with her toys']

simple_prompts = [
    'When Jack picked up a cat, Sarah picked up a dog',
    'Sarah saw a big scary dog on the driveway',
    'Sarah painted a picture of a zebra',
    'Jack and Sarah ate lunch and played together',
    'Sarah climbed a tree and fell down',
    'Jack and Sarah went to the park',
]

for prompt in continuous_prompts:
    print(model.generate(prompt, 20))


for prompt in simple_prompts:
    print(model.generate(prompt, 200))


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah was painting and it was very noisy when the wind blew. Suddenly, a big rainbow came to the town and it


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah was sleeping and dreamt of cherries. But when she woke up, she couldn't because it was so dark outside


  0%|          | 0/20 [00:00<?, ?it/s]

...

Jack and Sarah ate lunch and played together in the park. They were great friends. When they got tired, they sat in the sandbox and discussed what they were going to do next. Sarah said she wanted to find a shell and Jack said he wanted to spy a... can. 

Jack and Sarah looked around the park and spotted an interesting shell. It was round and big and full of lots of color. Jack and Sarah knew this was the perfect one for their game of finding the shell. 

They both got up and started looking for a special place. A beach nearby had the rock. They smiled at each other, happy that they had found something great and special on their day.
<|endoftext|>


  0%|          | 0/200 [00:00<?, ?it/s]

Sarah climbed a tree and fell down. She hurt her head and had to go to the hospital. She couldn't climb and play for a long time. Sarah was sad because she couldn't play with her friends.
<|endoftext|>


  0%|          | 0/200 [00:00<?, ?it/s]

Jack and Sarah went to the park to play. They saw a vendor selling ice cream. Jack wanted some, but Sarah said no. Jack got mad and started to shake. He wouldn't stop. In the end, Sarah stopped being mad and they decided to go to the park together to have fun. They played on the swings and the slide. It was such a happy day!
<|endoftext|>
